# **Homework assignment 3**


Course:  Business Forecasting

Prof:    Dr. White

<br>

Student: Gilian Koenders

Number:  59046858

Date:    11-13-2024



---







## Seting up the installation
In the code chunks below, the installation is prepared with installing Torchvision and the necessary libraries.




In [13]:
pip install torch torchvision torchaudio

In [14]:
# For reading data
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# For visualizing
import plotly.express as px
# For model building
import torch
import torch.nn as nn
import torch.nn.functional as F

#Import numpy
import numpy as np

## Data Loading

For the dataloading I could innitialy not make use of the file becuase I got a whole load of warnings, it said that there was a  chance of malware detected. Worked around it with the school computer, but don't have a fancy way of loading it, I had to place it into Google Colab temporary drive.


To make the model more robust, I added some image transformations, such as the random flips and slight rotations, and normalized the data. For the importing I used the IDX fiel format.

After the data is loaded I checked it based on a example. Herefore I had to rewrite as it is no longer searching for a 0, but 0=T-shirt. That gave me the confirmation that the data is in the correct shape.

In [15]:
!pip install python-mnist
import struct
import gzip
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from mnist import MNIST
import torchvision.transforms as transforms
from PIL import Image

class FashionMNIST(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.images_path = images_path
        self.labels_path = labels_path
        self.transform = transform
        self.images, self.labels = self.load_idx_files()

    def load_idx_files(self):
        with open(self.labels_path, 'rb') as lbpath:
            magic, num = struct.unpack(">II", lbpath.read(8))
            labels = torch.tensor(list(lbpath.read()), dtype=torch.long)

        with open(self.images_path, 'rb') as imgpath:
            magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
            images = torch.tensor(list(imgpath.read()), dtype=torch.uint8).view(num, rows, cols)

        return images, labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        image = Image.fromarray(image.numpy(), mode='L')

        if self.transform:
            image = self.transform(image)

        return image, label

# Transforms parameters
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(7),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])



Load the data manually in the Google Colab drive, the file is too big for me to store on the drive or on github.

In [16]:
# Load the FashionMNIST dataset
train_data = FashionMNIST('/content/train-images-idx3-ubyte.idx', '/content/train-labels-idx1-ubyte.idx', transform=transform)
test_data = FashionMNIST('/content/t10k-images-idx3-ubyte.idx', '/content/t10k-labels-idx1-ubyte.idx', transform=transform)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [20]:
# Define label names for better understanding
label_names = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}
# Check that our data look right when we sample
idx=1
image, label = train_data.__getitem__(idx)

# Rescale pixel values to 0-255 and convert to NumPy array
image_np = image.squeeze().numpy()  # Remove channel dimension if present
image_np = (image_np * 0.5 + 0.5) * 255  # Rescale to 0-255 range
image_np = image_np.astype(np.uint8)  # Ensure data type is uint8

print(f"This image is labeled a {label_names[label.item()]}")
px.imshow(image_np, color_continuous_scale="gray")

This image is labeled a T-shirt/top


##Now building the network
For the network , the parameters are set below. I added some extra layers, after some online search. This will give the neural network a better understanding, and can then better predict on unseen data.

To see if there are parametes print, I added a little line of code for it. This is used as check up if the model works fine. I left it inserted. After this code are the training parameters notated.

In [21]:
class FirstNet(nn.Module):
    def __init__(self):

      super(FirstNet, self).__init__()

      self.flatten = nn.Flatten()

      self.linear_relu_model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu_model(x)
        return output



In [22]:
# To see what the model contains I print the parameters
model = FirstNet()
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.0105,  0.0086, -0.0125,  ...,  0.0197,  0.0272,  0.0292],
        [-0.0061, -0.0258, -0.0278,  ...,  0.0311, -0.0098,  0.0290],
        [-0.0085, -0.0122,  0.0207,  ..., -0.0141,  0.0026, -0.0210],
        ...,
        [-0.0161,  0.0235, -0.0179,  ...,  0.0138,  0.0290, -0.0038],
        [-0.0171, -0.0173,  0.0140,  ...,  0.0104,  0.0260,  0.0125],
        [-0.0151, -0.0243,  0.0156,  ..., -0.0341, -0.0016, -0.0309]],
       requires_grad=True), Parameter containing:
tensor([ 3.0019e-02, -1.8864e-02, -2.5100e-02,  1.9676e-03,  3.4244e-02,
        -3.0937e-02, -6.5419e-03,  2.2159e-02, -2.4858e-02, -2.1569e-02,
         2.1972e-02, -2.6622e-02, -4.9197e-03,  2.4765e-02, -1.3105e-02,
        -2.3399e-02, -1.3346e-03, -3.0538e-02,  2.6711e-02, -4.9150e-03,
        -1.4489e-02, -1.6925e-02,  2.6323e-02, -2.4172e-02, -9.3698e-03,
        -2.8979e-02, -2.6857e-04, -2.1753e-02, -1.1536e-02,  1.1657e-02,
         2.3239e-02, -6.3269e-03, -2.5179e-03,  1.3316e

In [23]:
# Define some training parameters
learning_rate = 1e-2
batch_size = 64
epochs = 20

# Define our loss function
loss_fn = nn.CrossEntropyLoss()

In [33]:
#Implementing the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Training of Data

For the training part, only minor changes have been made because I had an error once. For the rest I left it as default. Only for the training part I eventually made some changes, to let it continue after 20 epochs

In [34]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    # Loop over batches via the dataloader
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Print progress update every few loops
        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [35]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Printing some output after a testing round
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [36]:
# Need to repeat the training process for each epoch.
#   In each epoch, the model will eventually see EVERY
#   observations in the data
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.287567  [   64/60000]
loss: 0.346154  [  704/60000]
loss: 0.184471  [ 1344/60000]
loss: 0.307112  [ 1984/60000]
loss: 0.334379  [ 2624/60000]
loss: 0.359173  [ 3264/60000]
loss: 0.266468  [ 3904/60000]
loss: 0.162830  [ 4544/60000]
loss: 0.362062  [ 5184/60000]
loss: 0.358872  [ 5824/60000]
loss: 0.259077  [ 6464/60000]
loss: 0.166177  [ 7104/60000]
loss: 0.227919  [ 7744/60000]
loss: 0.397168  [ 8384/60000]
loss: 0.252455  [ 9024/60000]
loss: 0.271041  [ 9664/60000]
loss: 0.154117  [10304/60000]
loss: 0.312020  [10944/60000]
loss: 0.212168  [11584/60000]
loss: 0.276612  [12224/60000]
loss: 0.305235  [12864/60000]
loss: 0.254099  [13504/60000]
loss: 0.247920  [14144/60000]
loss: 0.246068  [14784/60000]
loss: 0.215796  [15424/60000]
loss: 0.341520  [16064/60000]
loss: 0.428889  [16704/60000]
loss: 0.379980  [17344/60000]
loss: 0.271408  [17984/60000]
loss: 0.287836  [18624/60000]
loss: 0.337902  [19264/60000]
loss: 0.194428  [19904/60000]


In [37]:
# Decide if we are loading for predictions or more training
model.eval()
# - or -
#model.train()

# Make predictions
pred = model(test_data.__getitem__(1)[0]).argmax()
truth = test_data.__getitem__(1)[1]
print(f"This image is predicted to be a {pred}, and is labeled as {truth}")

This image is predicted to be a 2, and is labeled as 2


For the evaluation only giving me the predicted image was not enough, I had now clue how good my training data was on the testdata. So I trained it once, which gave me a poor outcome. Than I added the transformers and this improved my prediction on the test data drastically.

In [38]:
# Loading for predictions
model.eval()

# Initialize counters for correct predictions and total predictions
correct = 0
total = 0

# Iterate over the test dataset
for data, target in test_data:

    output = model(data)
    pred = output.argmax(dim=1, keepdim=True)

    correct += pred.eq(torch.tensor(target).unsqueeze(0)).sum().item()
    total += 1

# Calculate accuracy
accuracy = 100. * correct / total

print(f"Accuracy on test data: {accuracy:.2f}%")

<ipython-input-38-0c5b98c59a80>:14: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Accuracy on test data: 85.68%


Initially I had a score of 7.92%, but for my machine learning research I found someone that said that angles and especially rotations matter. This will let it 'learn' better to make better predictions for new data it has never seen before. So that was what I did, with my transformers.


##Saving the model


For saving the model I made some chances after I did not manage to save and export my file. Ultimately I think that it was not really necessary but I am able to see more about my data than only some basic information. With the export of the additional data I was better able to tell of my import was correct.

In [ ]:
# Save our model for later, so we can train more or make predictions
EPOCH = epochs

PATH = "model_BF.pt"

state = {
    'epoch': EPOCH,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'avarage_loss': {loss_fn},
    'validation_accuracy': {accuracy}, #added to see at import if it imports everything
}
torch.save(state, PATH)
from google.colab import files
files.download('model_BF.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Loading the model

For loading the model I made use of the github repository. In my Github the model.pt is placed and in here imported. When using the code of the lecture I had now clue of what was saved. So I changed to see which epoch it uses and what it latest accuracy was. With that I was able to understand what was going on in the file and which information it contained.

To load the data, you must run all the code chunks above, from the exception of the chapter 'Training of Data'. This because I placed the model and optimizer in there. I don't want hardcopy it or save it in a different way, as I also always need to load my .idx files because of there size.

In [26]:
import requests
import torch

filepath = 'https://github.com/Gilian2002/Assignment-3/raw/main/model_BF-4.pt' # Updated URL to raw file

# Download the file from GitHub
response = requests.get(filepath)

# Save the downloaded file locally
local_filepath = 'model_BF.pt'
with open(local_filepath, 'wb') as f:
    f.write(response.content)

# Now load the model using the local file path
state = torch.load(local_filepath, weights_only=False)

# Load model, optimizer, and epoch information
model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])
start_epoch = state['epoch']
last_val_accuracy = state.get('validation_accuracy', None)
last_avg_loss = state.get('avarage_loss', None)

print(f"Resuming from epoch {start_epoch}.")
print(f"Last saved epoch validation accuracy: {last_val_accuracy}")

model.train()

Resuming from epoch 20.
Last saved epoch validation accuracy: {86.51}


FirstNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): ReLU()
    (10): Linear(in_features=32, out_features=16, bias=True)
    (11): ReLU()
    (12): Linear(in_features=16, out_features=10, bias=True)
  )
)

## Testing the imported model
With the code below, which is the same as a little higher. A test is performed on how the model accuracy is on the testdata.

I have done this, because I had troubles on my testdata and kept losing my model somehow. So instead of only seeing what my image does I wanted to now what the % was on the test data. That was the reason why I left the code in so that you could see how my model performs on the test data.

In [29]:
model.eval()

# These are the counters for correct predictions and total predictions
correct = 0
total = 0

# Iterate over the test dataset
for data, target in test_data: #load the dataset in an earlier prompt
    output = model(data)
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(torch.tensor(target).unsqueeze(0)).sum().item()
    total += 1

# Calculate accuracy
accuracy = 100. * correct / total

print(f"Accuracy on test data: {accuracy:.2f}%")

<ipython-input-29-f59aebf1a84a>:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Accuracy on test data: 86.32%
